<div class="alert alert-block alert-success">
<b>Imported libraries</b>
</div>

In [1]:
import chart_studio
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import plotly.graph_objs as go
import chart_studio.plotly as py

from pandas_datareader import data

chart_studio.tools.set_config_file(world_readable = True, sharing = 'public')

C:\Users\Sif\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



<div class="alert alert-block alert-info">
<b>Load the stock historical data, transform in montly data and get percentage return</b>
</div>

In [2]:
end = dt.datetime.now()
start = dt.date(end.year - 5, end.month - 1, end.day)
start_rf = dt.date(start.year, start.month + 1, start.day)

In [3]:
# For a brazilian stock as FLRY3 you have to put a .SA so it will be FLRY3.SA
r = data.DataReader(['AMZN'], 'yahoo', start, end)['Adj Close']
r = r.resample('M').last().pct_change()
r.head()

Symbols,AMZN
Date,
2016-03-31,NaN
2016-04-30,0.111094
2016-05-31,0.095817
2016-06-30,-0.009920
2016-07-31,0.060353


In [4]:
rm = data.DataReader(['^GSPC'], 'yahoo', start, end)['Adj Close']
rm = rm.resample('M').last().pct_change()
rm.head()

Symbols,^GSPC
Date,
2016-03-31,NaN
2016-04-30,0.002699
2016-05-31,0.015325
2016-06-30,0.000911
2016-07-31,0.035610


<div class="alert alert-block alert-info">
<b>Percentage return of risk free investment with DGS10 = One Month Treasury Constant Maturity Rate</b>
</div>

In [5]:
rf = data.DataReader('DGS1MO', 'fred', start_rf, end)
rf = rf.resample('M').last()
rf = rf/100
rf.head()

,DGS1MO
DATE,
2016-04-30,0.0016
2016-05-31,0.0027
2016-06-30,0.0020
2016-07-31,0.0019
2016-08-31,0.0026


<div class="alert alert-block alert-info">
<b>Table for Capital Asset Pricing Model</b>
</div>

In [6]:
# dropna to drop the first month
df = pd.DataFrame({'r': r['AMZN'],
                   'rm': rm['^GSPC'], 
                   'rf': rf['DGS1MO'], 
                   'r - rf': r['AMZN'] - rf['DGS1MO'],
                   'rm - rf': rm['^GSPC'] - rf['DGS1MO']}).dropna()
df.head()

,r,rm,rf,r - rf,rm - rf
Date,,,,,
2016-04-30,0.111094,0.002699,0.0016,0.109494,0.001099
2016-05-31,0.095817,0.015325,0.0027,0.093117,0.012625
2016-06-30,-0.009920,0.000911,0.0020,-0.011920,-0.001089
2016-07-31,0.060353,0.035610,0.0019,0.058453,0.033710
2016-08-31,0.013640,-0.001219,0.0026,0.011040,-0.003819


In [7]:
X1 = sm.add_constant(df['rm - rf'])
model = sm.OLS(endog = df['r'], exog = X1, missing = 'drop')
results_req = model.fit()
results_req.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      r   R-squared:                       0.377
Model:                            OLS   Adj. R-squared:                  0.366
Method:                 Least Squares   F-statistic:                     35.69
Date:                Thu, 29 Apr 2021   Prob (F-statistic):           1.43e-07
Time:                        22:22:06   Log-Likelihood:                 82.503
No. Observations:                  61   AIC:                            -161.0
Df Residuals:                      59   BIC:                            -156.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0301      0.008      3.686      0.000       0.014       0.046
rm - rf        1.0856      0.182      5.974      0.000       0.722       1.449
==============================================================================
Omnibus:                        1.159   Durbin-Watson:                   1.795
Prob(Omnibus):                  0.560   Jarque-Bera (JB):                1.130
Skew:                           0.312   Prob(JB):                        0.568
Kurtosis:                       2.765   Cond. No.                         22.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<div class="alert alert-block alert-info">
<b>Expected monthly % required returns plot</b>
</div>

In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df['rm - rf'], y = df['r'], 
    name = 'Observed returns', 
    mode = 'markers',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(66, 133, 244)')))

fig.add_trace(go.Scatter(x = df['rm - rf'], y = results_req.fittedvalues,
    name = 'Predicted returns',
    mode = 'lines',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(234, 67, 53)')))

fig.update_layout(title_text = 'Returns to market risk premium (5 Years)', 
    title_x = 0.5,
    title_y = 0.8,
    autosize = True,
    legend = dict(
    x = 0, y = 1.5),
    xaxis_title = 'Monthly market risk premium',
    yaxis_title = 'Monthly AMZN return',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel = dict(
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_xaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

fig.update_yaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

py.iplot(fig, 
         filename = 'Linear regression of monthly return - Amazon (AMZN) and S&P 500', 
         auto_open = False)

$$R\ =\ α\ +\ \beta(R_m\ -\ R_f)\ + \varepsilon_t$$

In [9]:
X1 = sm.add_constant(df['rm - rf'])
model = sm.OLS(endog = df['r - rf'], exog = X1, missing = 'drop')
results_exce = model.fit()
results_exce.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 r - rf   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.395
Method:                 Least Squares   F-statistic:                     40.13
Date:                Thu, 29 Apr 2021   Prob (F-statistic):           3.57e-08
Time:                        22:22:11   Log-Likelihood:                 82.533
No. Observations:                  61   AIC:                            -161.1
Df Residuals:                      59   BIC:                            -156.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0194      0.008      2.377      0.021       0.003       0.036
rm - rf        1.1506      0.182      6.335      0.000       0.787       1.514
==============================================================================
Omnibus:                        1.854   Durbin-Watson:                   1.784
Prob(Omnibus):                  0.396   Jarque-Bera (JB):                1.579
Skew:                           0.392   Prob(JB):                        0.454
Kurtosis:                       2.925   Cond. No.                         22.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df['rm - rf'], y = df['r - rf'], 
    name = 'Observed excess returns', 
    mode = 'markers',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(255, 255, 145)')))

fig.add_trace(go.Scatter(x = df['rm - rf'], y = results_exce.fittedvalues, 
    name = 'Predicted excess returns',
    mode = 'lines',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(145, 255, 255)')))

fig.update_layout(title_text = 'Excess returns to market risk premium (5 Years)', 
    title_x = 0.5,
    title_y = 0.8,
    autosize = True,
    legend = dict(
    x = 0, y = 1.5),
    xaxis_title = 'Monthly market risk premium',
    yaxis_title = 'Monthly AMZN excess return',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel = dict(
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_xaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

fig.update_yaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

py.iplot(fig, 
         filename = 'Linear regression of monthly excess return - Amazon (AMZN) and S&P 500', 
         auto_open = False)

$$R\ -\ R_f =\ α\ +\ \beta(R_m\ -\ R_f)\ + \varepsilon_t$$